# Libraries

In [1]:
import sqlite3
import numpy as np
from collections import OrderedDict, namedtuple
from distutils.spawn import find_executable
import matplotlib.pyplot as plt

import sys
import subprocess
import pathlib
import tempfile
import importlib
import os
import time
import random
import pickle

import math

from configparser import ConfigParser

# from google.protobuf.json_format import MessageToDict

In [2]:
# DEVICE = 0

import os
import time
import pickle
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

import torch
torch.backends.cudnn.benchmark = True
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils import data

# torch.cuda.set_device(DEVICE)
torch.cuda.empty_cache() 
torch.backends.cudnn.benchmark=True

In [3]:
np.random.seed(0)
torch.manual_seed(0)

import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

# load the dataset

In [4]:
with open('scrimmage4_link_dataset.pickle', 'rb') as file:
    link_dataset = pickle.load(file)

In [5]:
train_data = link_dataset[:200] 
test_data = link_dataset[200:]

In [6]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, 2)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        lstm_out, _ = self.lstm(sentence.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

# Train Model 10 Epoch

In [7]:
EMBEDDING_DIM = 20
HIDDEN_DIM = 100
TAGSET_SIZE = 2
NUM_EPOCHS = 10

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE).cuda()
model.cuda()

loss_function = nn.NLLLoss().cuda()
optimizer = optim.Adam(model.parameters(), 0.001)

for epoch_idx in range(NUM_EPOCHS): 

    progress_training_epoch = tqdm(
        train_data, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_data), smoothing=.9)

    progress_validation_epoch = tqdm(
        test_data, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(test_data), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()
    for idx, (sentence, tags) in enumerate(progress_training_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        train_loss += loss * tags.size()[0]
        train_size += tags.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
            test_loss += loss * tags.size()[0]
            test_size += tags.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'test loss:{test_loss.item()/test_size: .5f}')

Epoch 1/10, Validation:   0%|                                    | 0/84 [00:00<?, ?it/s]

train loss: 0.56438, test loss: 0.60508


Epoch 2/10, Validation:   0%|                                    | 0/84 [00:00<?, ?it/s]

train loss: 0.54149, test loss: 0.58041


Epoch 3/10, Validation:   0%|                                    | 0/84 [00:00<?, ?it/s]

...

train loss: 0.51640, test loss: 0.56885


Epoch 9/10, Validation:   0%|                                    | 0/84 [00:00<?, ?it/s]

train loss: 0.50826, test loss: 0.56735


Epoch 9/10, Validation: 100%|███████████████████████████| 84/84 [14:28<00:00, 10.34s/it]

train loss: 0.50074, test loss: 0.57301


# Test

In [8]:
progress_validation_epoch = tqdm(
    test_data, 
    desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_data), smoothing=.9)

predict = []
target = []
model.eval()
with torch.no_grad():
    for idx, (sentence, tags) in enumerate(progress_validation_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        tag_scores = model(sentence)
        predict.append(tag_scores.argmax(dim=1).cpu().numpy())
        target.append(tags.cpu().numpy())
        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)

tp = predict[target==1].sum()
tn = (target==0).sum() - predict[target==0].sum()
fp = predict[target==0].sum()
fn = (target==1).sum() - predict[target==1].sum()

print(f'True Positive:{tp}, '
      f'True Negative:{tn}, '
      f'False Positive:{fp}, '
      f'False Negative:{fn}')

Epoch 9/10, Validation: 100%|███████████████████████████| 84/84 [02:38<00:00,  1.89s/it]

True Positive:1281279, True Negative:223380, False Positive:468038, False Negative:74233


In [9]:
accr = (tp+tn)/(tp+tn+fp+fn)
posaccr = tp/(tp+fp)
negaccr = tn/(tn+fn)
print(f'Accuracy:{accr}, '
      f'Positive Accuracy:{posaccr}, '
      f'Negative Accuracy:{negaccr}, '
      f'Weighted Accuracy:{0.5*(posaccr+negaccr)}')

Accuracy:0.7350808283624746, Positive Accuracy:0.7324452915051989, Negative Accuracy:0.750572051624089, Weighted Accuracy:0.741508671564644


# Bagging 9

In [10]:
with open('scrimmage4_link_dataset.pickle', 'rb') as file:
    link_dataset = pickle.load(file)

In [11]:
EMBEDDING_DIM = 20
HIDDEN_DIM = 100
TAGSET_SIZE = 2
NUM_EPOCHS = 6
model_list = []

for model_idx in range(9):
    
    train_data = []
    for link_idx in np.random.choice(200,size=200,replace=True):
        train_data.append(link_dataset[link_idx])
    
    model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE)
    model.cuda()
    loss_function = nn.NLLLoss().cuda()
    optimizer = optim.Adam(model.parameters(), 0.001)

    model.train()
    for epoch_idx in range(NUM_EPOCHS):  

        progress_training_epoch = tqdm(
            train_data, 
            desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Training',
            miniters=1, ncols=88, position=0,
            leave=True, total=len(train_data), smoothing=.9)

        progress_validation_epoch = tqdm(
            test_data, 
            desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
            miniters=1, ncols=88, position=0, 
            leave=True, total=len(test_data), smoothing=.9)

        train_loss = 0
        train_size = 0
        model.train()
        for idx, (sentence, tags) in enumerate(progress_training_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            model.zero_grad()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            loss.backward()
            optimizer.step()
            train_loss += loss * tags.size()[0]
            train_size += tags.size()[0]

        test_loss = 0
        test_size = 0
        model.eval()
        with torch.no_grad():
            for idx, (sentence, tags) in enumerate(progress_validation_epoch):
                sentence = sentence.cuda()
                tags = tags.cuda()
                tag_scores = model(sentence)
                loss = loss_function(tag_scores, tags) 
                test_loss += loss * tags.size()[0]
                test_size += tags.size()[0]

        print(f'train loss:{train_loss.item()/train_size: .5f}, '
              f'test loss:{test_loss.item()/test_size: .5f}')
        
    model_list.append(model)

Epoch 1/6, Validation:   0%|                                     | 0/84 [00:00<?, ?it/s]

train loss: 0.56260, test loss: 0.56916


Epoch 2/6, Validation:   0%|                                     | 0/84 [00:00<?, ?it/s]

train loss: 0.52249, test loss: 0.57076


Epoch 3/6, Validation:   0%|                                     | 0/84 [00:00<?, ?it/s]

...

train loss: 0.51313, test loss: 0.58475


Epoch 5/6, Validation:   0%|                                     | 0/84 [00:00<?, ?it/s]

train loss: 0.50629, test loss: 0.57919


Epoch 5/6, Validation: 100%|████████████████████████████| 84/84 [13:48<00:00,  9.86s/it]

train loss: 0.49545, test loss: 0.57831


# Test

In [12]:
predict_agg = []
target_agg = []

for model in model_list:
    
    progress_validation_epoch = tqdm(
        test_data, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(test_data), smoothing=.9)
    
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)
    
    predict_agg.append(predict)
    target_agg.append(target)
    
predict = (np.array(predict_agg).sum(axis=0) > 4.5)
target = target_agg[0]
tp = predict[target==1].sum()
tn = (target==0).sum() - predict[target==0].sum()
fp = predict[target==0].sum()
fn = (target==1).sum() - predict[target==1].sum()

print(f'True Positive:{tp}, '
      f'True Negative:{tn}, '
      f'False Positive:{fp}, '
      f'False Negative:{fn}')


Epoch 5/6, Validation: 100%|████████████████████████████| 84/84 [02:38<00:00,  1.89s/it]

True Positive:1258900, True Negative:253869, False Positive:437549, False Negative:96612


In [13]:
accr = (tp+tn)/(tp+tn+fp+fn)
posaccr = tp/(tp+fp)
negaccr = tn/(tn+fn)
print(f'Accuracy:{accr}, '
      f'Positive Accuracy:{posaccr}, '
      f'Negative Accuracy:{negaccr}, '
      f'Weighted Accuracy:{0.5*(posaccr+negaccr)}')

Accuracy:0.7390428593063759, Positive Accuracy:0.7420794848533614, Negative Accuracy:0.7243445436414527, Weighted Accuracy:0.733212014247407
